In [1]:
#run on GoogleDrive
from google.colab import drive
drive.mount('/content/drive/')
import xgboost as xgb
import lightgbm as lgb
import pandas as pd
import numpy as np
import warnings
import joblib
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import os
from datetime import timedelta
from tqdm import tqdm
import math
warnings.filterwarnings('ignore')
os.chdir('/content/drive/My Drive/m5-forecasting-accuracy')

Mounted at /content/drive/


In [2]:
def convert_obj_to_int(col):
    
    col = col.astype('category')
    col = col.cat.codes.astype('int64')
    col -= col.min()
    
    return col

#function used for getting the decimal part of the price
def get_last_2_digits(num):
    
    if str(num) != 'nan':
        new_num = num - int(num)
    else:
        new_num = num
        
    return new_num

#function used for reducing memory usage
def memory_reduction(df):
    
    init_mem = df.memory_usage().sum() / 1024**2 
    int_dtypes = [np.int8, np.int16, np.int32, np.int64]
    float_dtypes = [np.float16, np.float32, np.float64]
    
    for col in df.columns:
        if str(df[col].dtypes)[0:3] == 'int' or str(df[col].dtypes)[0:5] == 'float':
            min_value = df[col].min()
            max_value = df[col].max()
            if str(df[col].dtypes)[0:3] == 'int':
                for i in range(len(int_dtypes)):
                    if min_value > np.iinfo(int_dtypes[i]).min and max_value < np.iinfo(int_dtypes[i]).max:
                        df[col] = df[col].astype(int_dtypes[i])
                        break


            else:
                for i in range(len(float_dtypes)):
                    if min_value > np.finfo(float_dtypes[i]).min and max_value < np.finfo(float_dtypes[i]).max:
                        df[col] = df[col].astype(float_dtypes[i])
                        break
                        

    final_mem = df.memory_usage().sum() / 1024 ** 2
    pct_reduce = (init_mem - final_mem) / init_mem
    print('memory reduced to {:.2f} MB, a reduction of {:.2f}%'.format(final_mem, 100 * pct_reduce))

    return df

In [3]:
def preprocess_data():
    
    #load data
    price_df = pd.read_csv('sell_prices.csv')
    calendar_df = pd.read_csv('calendar.csv')
    df = pd.read_csv('sales_train_evaluation.csv')

    #get dtype dict of each dataframe
    calendar_dtypes = dict(calendar_df.dtypes)
    price_dtypes = dict(price_df.dtypes)
    df_dtypes = dict(df.dtypes)
    
    #unifies and changes the dtype of each dataframe
    for col in calendar_dtypes.keys():
        if calendar_dtypes[col] == 'object' and col not in ['date','d']:
            calendar_df[col] = convert_obj_to_int(calendar_df[col])
        #converts the dtype of 'date' column to 'datetime' for extracting time features later
        elif col == 'date':
            calendar_df[col] = pd.to_datetime(calendar_df[col])
            
    for col in price_dtypes.keys():
        if price_dtypes[col] == 'object':
            price_df[col] = convert_obj_to_int(price_df[col])

    for col in df_dtypes.keys():
        if df_dtypes[col] == 'object' and col != 'id':
            df[col] = convert_obj_to_int(df[col])
        elif str(df_dtypes[col])[0:3] == 'int':
            df[col] = df[col].astype('float32')
    
    #the goal is to predict the sales volume of the next 28 days, make it nan first
    last_day = 1941
    days_to_predict = 28
    for day in range(last_day+1, last_day+days_to_predict+1):
        df[f'd_{day}'] = np.nan
        
    #unpivots df from wide to long format
    value_vars = [col for col in df.columns if str(col)[0:2]=='d_']
    id_vars = df.columns.difference(value_vars)
    df = pd.melt(df, 
             id_vars = id_vars,
             value_vars = value_vars,
             var_name = 'd',
             value_name = 'sales')
    
    #merges df, calendar_df, price_df together
    df = pd.merge(df, calendar_df, how='left', on='d')
    df = pd.merge(df, price_df, how='left', on=['store_id', 'item_id', 'wm_yr_wk'])

    return df

In [4]:
%%time
df = preprocess_data()
df = memory_reduction(df)

memory reduced to 3206.20 MB, a reduction of 69.57%
CPU times: user 2min 26s, sys: 6.43 s, total: 2min 32s
Wall time: 2min 35s


In [5]:
df.columns

Index(['cat_id', 'dept_id', 'id', 'item_id', 'state_id', 'store_id', 'd',
       'sales', 'date', 'wm_yr_wk', 'weekday', 'wday', 'month', 'year',
       'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2',
       'snap_CA', 'snap_TX', 'snap_WI', 'sell_price'],
      dtype='object')

In [6]:
def create_features(df):
    
    # id-wise simple shift features
    shift_params = [7, 14, 21, 28, 35]
    for s in shift_params:
        df[f'feature_lag_t{s}'] = df.groupby(['id'])['sales'].apply(lambda x:x.shift(s)).astype('float16')
    print('part 1 completed!')

    # id-wise shift rolling mean and std features
    rolling_params = [7, 14, 30, 45]
    for s in shift_params:
        for r in rolling_params:
            df['rolling_mean'+str(r)+'_shift'+str(s)] = df.groupby(['id'])['sales'].apply(lambda x: x.shift(s).rolling(r).mean()).astype('float16')
            df['rolling_std'+str(r)+'_shift'+str(s)] = df.groupby(['id'])['sales'].apply(lambda x: x.shift(s).rolling(r).std()).astype('float16')
    df = df[(df['date'] >= '2016-04-22') | (pd.notna(df['rolling_mean45_shift35']))]
    print('part 2 completed!')

    # store-wise mean and std features
    df['store_id_mean'] = df.groupby(['store_id'])['sales'].transform('mean').astype('float16')
    df['store_id_std'] = df.groupby(['store_id'])['sales'].transform('std').astype('float16')
    print('part 3 completed!')

    # item_id-wise mean and std features
    df['item_id_mean'] = df.groupby(['item_id'])['sales'].transform('mean').astype('float16')
    df['item_id_std'] = df.groupby(['item_id'])['sales'].transform('std').astype('float16')
    print('part 4 completed!')

    # time features
    df['day_week'] = getattr(df['date'].dt, 'dayofweek').astype('int8')
    df['day_month'] = getattr(df['date'].dt, 'day').astype('int8')
    df['week_month'] = df['day_month'].apply(lambda x: math.ceil(x/7)).astype('int8')
    df['week_year'] = getattr(df['date'].dt, 'weekofyear').astype('int8')
    df['month'] = getattr(df['date'].dt, 'month').astype('int8')
    df['quarter'] = getattr(df['date'].dt, 'quarter').astype('int8')
    df['year'] = getattr(df['date'].dt, 'year').astype('int16')
    df['year'] = (df['year']-df['year'].min()).astype('int8')
    df['if_weekend'] = (df['day_week'] >= 5).astype('int8')
    print('part 5 completed!')
    
    # price features
    price_params = ['max','min','median','mean','std']
    for p in price_params:
        df[f'price_{p}'] = df.groupby(['id'])['sell_price'].transform(p)
    df['price_momentum_day'] = df['sell_price'] / df.groupby(['id'])['sell_price'].apply(lambda x:x.shift(1))
    df['price_momentum_week'] = df['sell_price'] / df.groupby(['id'])['sell_price'].apply(lambda x:x.shift(7))
    df['price_momentum_month'] = df['sell_price'] / df.groupby(['id','month'])['sell_price'].transform('mean')
    df['price_momentum_year'] = df['sell_price'] / df.groupby(['id','year'])['sell_price'].transform('mean')
    df['price_last_2_digits'] = df['sell_price'].apply(lambda x: get_last_2_digits(x))
    print('part 6 completed!')

    
    return df

In [7]:
%%time
df = create_features(df)

part 1 completed!
part 2 completed!
part 3 completed!
part 4 completed!
part 5 completed!
part 6 completed!
CPU times: user 33min 1s, sys: 22.2 s, total: 33min 23s
Wall time: 33min 23s


In [11]:
df.tail(5)

,cat_id,dept_id,id,item_id,state_id,store_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,feature_lag_t7,feature_lag_t14,feature_lag_t21,feature_lag_t28,feature_lag_t35,rolling_mean7_shift7,rolling_std7_shift7,rolling_mean14_shift7,rolling_std14_shift7,rolling_mean30_shift7,rolling_std30_shift7,rolling_mean45_shift7,rolling_std45_shift7,rolling_mean7_shift14,rolling_std7_shift14,rolling_mean14_shift14,rolling_std14_shift14,rolling_mean30_shift14,...,rolling_mean30_shift21,rolling_std30_shift21,rolling_mean45_shift21,rolling_std45_shift21,rolling_mean7_shift28,rolling_std7_shift28,rolling_mean14_shift28,rolling_std14_shift28,rolling_mean30_shift28,rolling_std30_shift28,rolling_mean45_shift28,rolling_std45_shift28,rolling_mean7_shift35,rolling_std7_shift35,rolling_mean14_shift35,rolling_std14_shift35,rolling_mean30_shift35,rolling_std30_shift35,rolling_mean45_shift35,rolling_std45_shift35,store_id_mean,store_id_std,item_id_mean,item_id_std,day_week,day_month,week_month,week_year,quarter,if_weekend,price_max,price_min,price_median,price_mean,price_std,price_momentum_day,price_momentum_week,price_momentum_month,price_momentum_year,price_last_2_digits
60034805,0,2,FOODS_3_823_WI_3_evaluation,1432,2,9,d_1969,NaN,2016-06-19,11621,3,2,6,5,17,4,3,1,0,0,0,2.980469,NaN,NaN,NaN,1.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.571289,0.534668,0.643066,0.841797,0.633301,0.808594,0.533203,0.786133,0.714355,1.112305,0.714355,0.914062,0.533203,0.819336,0.444336,0.785156,1.105469,4.039062,0.783203,1.667969,6,19,3,24,2,1,2.980469,2.480469,2.880859,2.814453,0.163468,1.0,1.0,1.030273,1.024414,0.980469
60034806,0,2,FOODS_3_824_WI_3_evaluation,1433,2,9,d_1969,NaN,2016-06-19,11621,3,2,6,5,17,4,3,1,0,0,0,2.480469,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.285645,0.488037,0.142822,0.363037,0.300049,0.535156,0.288818,0.505371,0.000000,0.000000,0.214233,0.579102,0.233276,0.503906,0.333252,0.563965,1.105469,4.039062,0.421143,0.933594,6,19,3,24,2,1,2.679688,2.000000,2.679688,2.509766,0.258118,1.0,1.0,0.991211,1.116211,0.480469
60034807,0,2,FOODS_3_825_WI_3_evaluation,1434,2,9,d_1969,NaN,2016-06-19,11621,3,2,6,5,17,4,3,1,0,0,0,3.980469,NaN,NaN,NaN,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.856934,0.899902,1.000000,1.109375,0.766602,0.897461,0.822266,1.006836,1.142578,1.344727,0.785645,1.050781,0.700195,0.876953,0.888672,1.091797,1.105469,4.039062,0.688477,1.145508,6,19,3,24,2,1,4.378906,3.980469,3.980469,4.121094,0.190039,1.0,1.0,0.957520,1.000000,0.980469
60034808,0,2,FOODS_3_826_WI_3_evaluation,1435,2,9,d_1969,NaN,2016-06-19,11621,3,2,6,5,17,4,3,1,0,0,0,1.280273,NaN,NaN,NaN,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.857422,2.267578,1.428711,1.650391,1.366211,1.299805,1.200195,1.235352,1.000000,0.577148,1.142578,0.663086,1.099609,0.922852,1.044922,0.903320,1.105469,4.039062,0.673828,1.271484,6,19,3,24,2,1,1.280273,1.280273,1.280273,1.280273,0.000000,1.0,1.0,1.000000,1.000000,0.280273
60034809,0,2,FOODS_3_827_WI_3_evaluation,1436,2,9,d_1969,NaN,2016-06-19,11621,3,2,6,5,17,4,3,1,0,0,0,1.000000,NaN,NaN,NaN,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.714844,1.975586,1.642578,1.823242,1.166992,1.463867,0.799805,1.307617,0.571289,0.786621,1.000000,0.960938,0.533203,0.819336,0.688965,0.996094,1.105469,4.039062,0.651855,2.099609,6,19,3,24,2,1,1.000000,1.000000,1.000000,1.000000,0.000000,1.0,1.0,1.000000,1.000000,0.000000


In [21]:
features_to_exclude = ['id','sales','date','d','weekday','wm_yr_wk','state_id','store_id','event_name_1','event_type_1','event_name_2','event_type_2']
features_to_use = [i for i in df.columns if i not in features_to_exclude]

In [37]:
def xgb_model(store_id, df):

    df = df[df['store_id'] == store_id]
    X_train = df[df['date'] <= '2016-04-22']
    Y_train = X_train['sales']
    X_valid = df[(df['date'] > '2016-04-22') & (df['date'] <= '2016-05-22')]
    Y_valid = X_valid['sales']
    
    train_set = xgb.DMatrix(X_train[features_to_use],label=Y_train)
    valid_set = xgb.DMatrix(X_valid[features_to_use],label=Y_valid)
    watch_list = [(train_set, 'train'), (valid_set, 'valid')]
    param = {'max_depth' : 6, 
             'eta' : 0.047, 
             'silent' : 1, 
             'objective': 'reg:linear',
             'subsample': 0.8,
             'colsample_bytree' : 0.5, 
             'min_child_weight' : 7,
             'verbose_eval' : 20}
    model = xgb.train(param, train_set, num_boost_round=500, early_stopping_rounds=20, evals=watch_list)

    return model

In [ ]:
def lgb_model(store_id, df):

    df = df[df['store_id'] == store_id]
    X_train = df[df['date'] <= '2016-04-22']
    Y_train = X_train['sales']
    X_valid = df[(df['date'] > '2016-04-22') & (df['date'] <= '2016-05-22')]
    Y_valid = X_valid['sales']

    cat_featuress = ['item_id', 'dept_id', 'cat_id'] + ["event_name_1", "event_name_2", "event_type_1", "event_type_2"] + ['snap_CA', 'snap_WI', 'snap_TX']

    train_set = lgb.Dataset(X_train[features_to_use], 
                            Y_train, 
                            categorical_feature = cat_features, 
                            free_raw_data = False)
    valid_set = lgb.Dataset(X_valid[features_to_use], 
                            Y_valid, 
                            categorical_feature = cat_features, 
                            free_raw_data = False)
    params = {
          'boosting_type': 'gbdt',
          'objective': 'tweedie',
          'tweedie_variance_power': 1.1,
          'metric': 'rmse',
          'subsample': 0.85,
          'subsample_freq': 1,
          'learning_rate': 0.03,
          'num_leaves': 2**11-1,
          'min_data_in_leaf': 2**12-1,
          'feature_fraction': 0.6,
          'max_bin': 100,
          'boost_from_average': False,
          'verbose': -1,
          'seed': 42}
    
    model = lgb.train(params, train_set, num_boost_round=100, early_stopping_rounds=100, valid_sets = [train_set, valid_set], verbose_eval=100)

    return model

In [ ]:
for i in range(10):
    print('model',i+1)
    model_name = 'model' + str(i) + '.sav' 
    model = xgb_model(i, df)
    joblib.dump(model, model_name)

model 1


In [14]:
def add_features(df):
    df['feature_lag_t7'] = df.groupby(['id'])['sales'].apply(lambda x:x.shift(7)).astype('float16')
    df['feature_lag_t14'] = df.groupby(['id'])['sales'].apply(lambda x:x.shift(14)).astype('float16')
    df['feature_lag_t21'] = df.groupby(['id'])['sales'].apply(lambda x:x.shift(21)).astype('float16')

    new_shift_params =[7, 14, 21]
    new_rolling_params = [7, 14, 30, 45]

    for s in new_shift_params:
        for r in new_rolling_params:
            df['rolling_mean'+str(r)+'_shift'+str(s)] = df.groupby(['id'])['sales'].apply(lambda x: x.shift(s).rolling(r).mean()).astype('float16')
            df['rolling_std'+str(r)+'_shift'+str(s)] = df.groupby(['id'])['sales'].apply(lambda x: x.shift(s).rolling(r).std()).astype('float16')

    return df

#predict the sales using the model
def make_predictions(df, model, store_id):
    df = df[df['store_id'] == store_id]
    test_set_week1 = df[(df['date'] > '2016-05-22') & (df['date'] <= '2016-05-29')] 
    df.loc[(df['date'] > '2016-05-22') & (df['date'] <= '2016-05-29'), 'sales'] = model.predict(test_set_week1[features_to_use])

    df = add_features(df)
    test_set_week2 = df[(df['date'] > '2016-05-29') & (df['date'] <= '2016-06-25')] 
    df.loc[(df['date'] > '2016-05-29') & (df['date'] <= '2016-06-05'), 'sales'] = model.predict(test_set_week2[features_to_use])

    df = add_features(df)
    test_set_week3 = df[(df['date'] > '2016-06-05') & (df['date'] <= '2016-06-12')] 
    df.loc[(df['date'] > '2016-06-05') & (df['date'] <= '2016-06-12'), 'sales'] = model.predict(test_set_week3[features_to_use])

    df = add_features(df)
    test_set_week4 = df[(df['date'] > '2016-06-12') & (df['date'] <= '2016-06-19')] 
    df.loc[(df['date'] > '2016-06-12') & (df['date'] <= '2016-06-19'), 'sales'] = model.predict(test_set_week4[features_to_use])

    return df[(df['date'] > '2016-05-22') & (df['date'] <= '2016-06-19')] 

30490

In [ ]:
for i in range(1, 10+1):
    model_name = 'model' + str(i) + '.sav'
    model = joblib.load(model_name)

    if i == 1:
        result = make_predictions(df, model, i)
    else:
        result = pd.concat([result, make_predictions(df, model, i)], axis=0)

In [32]:
import xgboost as xgb
#regressor = xgb.XGBRegressor(n_estimators=20, max_depth=6, subsample=0.8, learning_rate=0.04, min_child_weight=3)
regressor = xgb.XGBRegressor(n_estimators=30, max_depth=6, learning_rate=0.047, min_child_weight=7, subsample=0.8)
param = {'colsample_bytree' : [0.4, 0.5, 0.6, 0.7, 0.8, 0.9]}

#n_iter, int, default=10
#Number of parameter settings that are sampled. n_iter trades off runtime vs quality of the solution.
grid = RandomizedSearchCV(regressor, 
                          param, 
                          cv=3, 
                          scoring='neg_root_mean_squared_error', 
                          n_iter=6,
                          verbose=1)

In [33]:
%%time
df0 = df[df['store_id']==0]
X_train = df0[(df0['date'] <= '2016-04-22') & (df0['date'] >= '2015-04-22')]
Y_train = X_train['sales']
tqdm(grid.fit(X_train[features_to_use], Y_train))

Fitting 3 folds for each of 6 candidates, totalling 18 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[12:09:24] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:10:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:11:49] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:13:02] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:14:25] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:15:49] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:17:12] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:18:47] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

[Parallel(n_jobs=1)]: Done  18 out of  18 | elapsed: 30.1min finished


[12:39:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


0it [00:00, ?it/s]

CPU times: user 32min 25s, sys: 642 ms, total: 32min 26s
Wall time: 32min 26s


In [34]:
grid.best_params_

{'colsample_bytree': 0.5}

In [35]:
grid.cv_results_

{'mean_fit_time': array([ 71.91131719,  82.32553109,  94.33311383, 104.37431343,
        116.46956921, 126.49758959]),
 'mean_score_time': array([1.0800763 , 1.05936877, 1.05722809, 1.06534958, 1.06478715,
        1.058484  ]),
 'mean_test_score': array([-2.34680851, -2.34304396, -2.34305406, -2.3453606 , -2.34386269,
        -2.34347248]),
 'param_colsample_bytree': masked_array(data=[0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'colsample_bytree': 0.4},
  {'colsample_bytree': 0.5},
  {'colsample_bytree': 0.6},
  {'colsample_bytree': 0.7},
  {'colsample_bytree': 0.8},
  {'colsample_bytree': 0.9}],
 'rank_test_score': array([6, 1, 2, 5, 4, 3], dtype=int32),
 'split0_test_score': array([-2.48316026, -2.47757459, -2.48291469, -2.47325206, -2.4756701 ,
        -2.47603512]),
 'split1_test_score': array([-2.33144665, -2.32621503, -2.32586908, -2.34050035, -2.33135986,
        -